In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784',version = 1)

In [2]:
X,y = mnist["data"],mnist["target"]
#taking only the first 1000 to save time on training

X = mnist["data"][:1000]
y = mnist["target"][:1000]

In [3]:
from sklearn.model_selection import train_test_split
Xtrain,xtest,Ytrain,ytest = train_test_split(X,y,test_size = 0.15,random_state = 42)

In [4]:
Xtrain.shape

(850, 784)

In [6]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

In [17]:
import numpy as np

err = 0
accuracy_decreasing = 0
skf = StratifiedKFold(n_splits = 5, random_state = 42)
for i in range(1,200):
    print('Run :{}'.format(i))
    bag_clf = BaggingClassifier(DecisionTreeClassifier(), bootstrap=True, n_estimators= i, n_jobs= -1, max_samples=100)
    score = np.mean(cross_val_score(bag_clf, Xtrain,Ytrain,cv=skf, scoring = 'f1_weighted'))
    if score > err:
        accuracy_decreasing = 0
        err = score
        print(err)
    else:
        accuracy_decreasing = accuracy_decreasing + 1
        if accuracy_decreasing > 5:
            print(i - 5)
            break


Run :1
0.4396890841972585
Run :2
Run :3
0.48391106590697525
Run :4
0.5674489063072675
Run :5
0.6152505072548737
Run :6
0.650572402693386
Run :7
0.6569316910766178
Run :8
0.6903240470005176
Run :9
Run :10
0.6976031358250672
Run :11
Run :12
Run :13
0.7028771003477753
Run :14
0.7558695157476543
Run :15
Run :16
Run :17
Run :18
Run :19
Run :20
15


In [41]:
bag_clf_final = BaggingClassifier(DecisionTreeClassifier(), bootstrap=True, n_estimators= 500, n_jobs= -1, max_samples=100)
bag_clf_final.fit(Xtrain,Ytrain)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort=False,
                                                        random_state=None,
                                                        splitter='best'),
    

In [42]:
ypred_train = bag_clf_final.predict(Xtrain)

In [43]:
from sklearn.metrics import confusion_matrix
confusion_matrix_train = confusion_matrix(Ytrain,ypred_train)
confusion_matrix_train

array([[ 83,   0,   0,   0,   0,   0,   1,   0,   0,   0],
       [  0, 101,   0,   0,   0,   0,   1,   1,   0,   0],
       [  1,  10,  63,   0,   2,   0,   0,   6,   0,   0],
       [  1,   1,   0,  70,   2,   0,   0,   1,   0,   1],
       [  0,   1,   0,   0,  92,   0,   2,   1,   0,   0],
       [  0,   0,   0,   1,   1,  73,   1,   1,   1,   0],
       [  1,   1,   0,   0,   2,   0,  71,   1,   0,   0],
       [  1,   1,   0,   0,   2,   0,   0,  93,   0,   1],
       [  0,   3,   0,   1,   0,   1,   0,   0,  71,   0],
       [  1,   2,   0,   0,   1,   1,   1,   5,   0,  70]], dtype=int64)

In [25]:
for i in range(0,10):
    precision = confusion_matrix_train[i,i]/np.sum(confusion_matrix_train[:,i])
    recall = confusion_matrix_train[i,i]/np.sum(confusion_matrix_train[i,:])
    print('{},Precision:{},Recall:{}'.format(i,precision,recall))

0,Precision:0.8695652173913043,Recall:0.9523809523809523
1,Precision:0.7050359712230215,Recall:0.9514563106796117
2,Precision:0.7457627118644068,Recall:0.5365853658536586
3,Precision:0.7808219178082192,Recall:0.75
4,Precision:0.8282828282828283,Recall:0.8541666666666666
5,Precision:0.6823529411764706,Recall:0.7435897435897436
6,Precision:0.8840579710144928,Recall:0.8026315789473685
7,Precision:0.7758620689655172,Recall:0.9183673469387755
8,Precision:0.8703703703703703,Recall:0.618421052631579
9,Precision:0.90625,Recall:0.7160493827160493


In [44]:
for i in range(0,10):
    precision = confusion_matrix_train[i,i]/np.sum(confusion_matrix_train[:,i])
    recall = confusion_matrix_train[i,i]/np.sum(confusion_matrix_train[i,:])
    print('{},Precision:{},Recall:{}'.format(i,precision,recall))

0,Precision:0.9431818181818182,Recall:0.9880952380952381
1,Precision:0.8416666666666667,Recall:0.9805825242718447
2,Precision:1.0,Recall:0.7682926829268293
3,Precision:0.9722222222222222,Recall:0.9210526315789473
4,Precision:0.9019607843137255,Recall:0.9583333333333334
5,Precision:0.9733333333333334,Recall:0.9358974358974359
6,Precision:0.922077922077922,Recall:0.9342105263157895
7,Precision:0.8532110091743119,Recall:0.9489795918367347
8,Precision:0.9861111111111112,Recall:0.9342105263157895
9,Precision:0.9722222222222222,Recall:0.8641975308641975


In [45]:
from sklearn.metrics import accuracy_score
accuracy_score(Ytrain, ypred_train)

0.9258823529411765

In [46]:
ypred_test = bag_clf_final.predict(xtest)
confusion_matrix_test = confusion_matrix(ytest,ypred_test)
confusion_matrix_test

array([[12,  0,  0,  0,  0,  0,  0,  1,  0,  0],
       [ 0, 13,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  3,  9,  1,  1,  0,  1,  2,  0,  0],
       [ 1,  1,  0, 13,  0,  1,  0,  1,  0,  0],
       [ 0,  0,  0,  0,  9,  0,  0,  0,  0,  0],
       [ 0,  0,  1,  1,  0, 11,  0,  0,  0,  1],
       [ 0,  2,  0,  0,  2,  0, 14,  0,  0,  0],
       [ 0,  3,  0,  0,  1,  0,  0, 15,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  1,  0,  9,  0],
       [ 1,  0,  0,  1,  2,  0,  0,  1,  0, 14]], dtype=int64)

In [47]:
for i in range(0,10):
    precision = confusion_matrix_test[i,i]/np.sum(confusion_matrix_test[:,i])
    recall = confusion_matrix_test[i,i]/np.sum(confusion_matrix_test[i,:])
    print('{},Precision:{},Recall:{}'.format(i,precision,recall))

0,Precision:0.8571428571428571,Recall:0.9230769230769231
1,Precision:0.5909090909090909,Recall:1.0
2,Precision:0.9,Recall:0.5294117647058824
3,Precision:0.8125,Recall:0.7647058823529411
4,Precision:0.6,Recall:1.0
5,Precision:0.8461538461538461,Recall:0.7857142857142857
6,Precision:0.875,Recall:0.7777777777777778
7,Precision:0.75,Recall:0.7894736842105263
8,Precision:1.0,Recall:0.8181818181818182
9,Precision:0.9333333333333333,Recall:0.7368421052631579
